In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import logging
import pickle
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc
from importlib import reload
import pyreadstat
import disease_network_funs as fun_py
import re

In [2]:
reload(fun_py)

<module 'disease_network_funs' from '/home/hashjamm/codes/disease_network/disease_network_funs.py'>

In [3]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"
edge_pids_path = "/home/hashjamm/results/disease_network/edge_pids/"
matched_path = "/home/hashjamm/project_data/disease_network/sas_files/matched/"
ctable_path = "/home/hashjamm/results/disease_network/ctables/"
pids_info_path = "/home/hashjamm/results/disease_network/"
final_results_path = "/home/hashjamm/results/disease_network/final_results/"

In [4]:
node_pids_info = pd.read_csv(f'{pids_info_path}node_pids_info.csv')

In [5]:
node_pids_info

,node_code,width,height,sex_1_counts,sex_2_counts,age_0_counts,age_1_counts,age_2_counts,age_3_counts,age_4_counts,...,sex_2_ctrb_1_counts,sex_2_ctrb_2_counts,sex_2_ctrb_3_counts,sex_2_ctrb_4_counts,sex_2_ctrb_5_counts,sex_2_ctrb_6_counts,sex_2_ctrb_7_counts,sex_2_ctrb_8_counts,sex_2_ctrb_9_counts,sex_2_ctrb_10_counts
0,J03,1.000000,1.000000,41118,51040,1227,7231,11844,7787,5556,...,3165,3198,3706,4041,4719,5337,6200,6945,6932,6680
1,J20,0.999158,0.999158,33557,44137,1392,7547,8334,5408,3702,...,2885,2676,3076,3551,4110,4759,5237,6008,5889,5823
2,J06,0.998315,0.998315,30228,39585,1318,7861,7637,4603,3452,...,2585,2400,2858,3271,3662,4201,4824,5166,5258,5252
3,J02,0.997473,0.997473,27607,36329,1561,7785,7644,4575,3204,...,2218,2092,2446,2923,3262,3855,4456,4892,5164,4934
4,J00,0.996630,0.996630,27512,35232,1308,7750,6915,4203,3031,...,2446,2197,2525,2972,3344,3615,4196,4626,4698,4474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,T37,0.014322,0.167650,8,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1183,T54,0.038753,0.109520,5,5,0,0,1,0,0,...,1,0,0,1,0,0,1,0,2,0
1184,T75,0.014322,0.167650,8,2,0,0,2,0,1,...,0,0,0,0,1,0,1,0,0,0
1185,T94,0.076664,0.085088,3,7,0,1,0,3,0,...,3,0,2,0,1,0,1,0,0,0


In [5]:
cause_pids_info = fun_py.colname_type_setting(node_pids_info, 1, 'cause')
cause_pids_info = cause_pids_info.rename(columns={'node_code':'cause_abb'})

In [6]:
outcome_pids_info = fun_py.colname_type_setting(node_pids_info, 1, 'outcome')
outcome_pids_info = outcome_pids_info.rename(columns={'node_code':'outcome_abb'})

In [8]:
for i in range(1, 11):
    edge_pids_info = pd.read_csv(f'{pids_info_path}edge_pids_info_{i}.csv')
    stat_cut_result = pd.read_csv(f'{ctable_path}stat_cut_result_{i}.csv')
    
    edge_pids_info = fun_py.colname_type_setting(edge_pids_info, 2, 'edge')
    
    final_result =\
    pd.merge(stat_cut_result, edge_pids_info, on=['cause_abb', 'outcome_abb'], how='inner').reset_index(drop=True)
    
    final_result =\
    pd.merge(final_result, cause_pids_info, on=['cause_abb'], how='inner').reset_index(drop=True)
    
    final_result =\
    pd.merge(final_result, outcome_pids_info, on=['outcome_abb'], how='inner').reset_index(drop=True)
    
    cis_cut_final_result = final_result[final_result['rr_lower_cis'] > 1]
    
    final_result.to_csv(f"{final_results_path}final_result_{i}.csv", index=False)
    cis_cut_final_result.to_csv(f"{final_results_path}cis_cut_final_result_{i}.csv", index=False)